In [1]:
import locale

# locale.getpreferredencoding = lambda: "UTF-8" # wandb throws without this

!pip install datasets accelerate bitsandbytes transformers peft wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForLanguageModeling
from datasets import load_dataset
from functools import cache
import torch

device = "cuda"
# base_model_id = "mistralai/Mistral-7B-v0.1"
base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

@cache
def get_model_and_tokenizer():
  model = AutoModelForCausalLM.from_pretrained(
      base_model_id,
      quantization_config=bnb_config
  )

  tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side="left")
  tokenizer.add_eos_token = True
  tokenizer.pad_token = tokenizer.eos_token

  return model, tokenizer

model, tokenizer = get_model_and_tokenizer()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def tokenize(prompt):
  result = tokenizer(
      prompt,
      max_length=2000,
      truncation=True,
      return_tensors=None,
      padding="max_length"
  )
  result["labels"] = result["input_ids"].copy()
  return result

def generate_and_tokenize_prompt(example):
    full_prompt =f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an imaginative short children's story about a boy called Haris and his cat called Howie. The story must be between 400 and 500 words.

### Response:
{example["output"]}
"""
    return tokenize(full_prompt)


In [5]:
import pickle, os

redo_tokenization = True
train_tokens_file_path = "tokenized-train.pkl"
test_tokens_file_path = "tokenized-test.pkl"
dataset = load_dataset("json", data_files="book_data.jsonl")

train_dataset = dataset["train"]
train_test_split = train_dataset.train_test_split(test_size=0.1)
test_dataset = train_test_split["test"]

def load_tokens_from_file(filepath=None):
  with open(filepath, 'rb') as file:
      return pickle.load(file)

def dump_tokens_to_file(dataset=None, filepath=None):
  if os.path.exists(filepath) == False or redo_tokenization == True:
    with open(filepath, 'wb') as file:
        pickle.dump(dataset.map(generate_and_tokenize_prompt), file)

dump_tokens_to_file(train_dataset, train_tokens_file_path)
dump_tokens_to_file(test_dataset, test_tokens_file_path)

tokenized_train_dataset = load_tokens_from_file(train_tokens_file_path)
tokenized_test_dataset = load_tokens_from_file(test_tokens_file_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [6]:

print(tokenized_train_dataset)
print(tokenized_test_dataset)

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 97
})
Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})


In [ ]:
### Test the base model before fine-tuning

# eval_prompt = f"""Below is an instruction that describes a task. Write a single response that appropriately completes the request.

# ### Instruction:
# Write an imaginative short children's story about a boy called Haris and his cat called Howie. The story must be no shorter than 500 words and no longer than 750 words.

# ### Response:
# """

eval_prompt = "Write an imaginative short children's story about a boy called Haris and his cat called Howie. The story must be no shorter than 500 words and no longer than 750 words"

model_input = tokenizer(
    eval_prompt,
    return_tensors="pt",
    max_length=2000,
    truncation=True
).to(device)

print(model_input)
model.eval()

generated_ids = model.generate(**model_input, do_sample=True, max_new_tokens=5000)

with torch.no_grad():
    print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])
#     print(tokenizer.decode(model.generate(**model_input)[0], skip_special_tokens=True))

In [7]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

model.train() # put model back into training mode
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
# print(model)

In [7]:
from datetime import datetime
import wandb

project_name = "mistral-books-finetune"
output_dir = "./" + project_name

wandb.login()
os.environ["WANDB_PROJECT"] = project_name

if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    args=TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=20,
        max_steps=100,
        learning_rate=3e-5,
        # learning_rate=2.5e-5, # want ~10x smaller than the Mistral learning rate
        logging_steps=1,
        fp16=True, # V100
        # bf16=True, # A100 only
        optim="paged_adamw_8bit",
        push_to_hub=True,
        # weight_decay=0.01,
        logging_dir="./logs",        # Directory for storing logs
        # save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=10,                # Save checkpoints every 10 steps
        # evaluation_strategy="steps", # Evaluate the model every logging step
        # eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        # do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{project_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

print("Training...")

train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
print(metrics)
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Training...


wandb: Currently logged in as: jamesarobbo. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.202100
2,1.203400
3,1.212700
4,1.213200
5,1.198500
6,1.196700
7,1.200300
8,1.189200
9,1.188800
10,1.176700


Step,Training Loss
1,1.202100
2,1.203400
3,1.212700
4,1.213200
5,1.198500
6,1.196700
7,1.200300
8,1.189200
9,1.188800
10,1.176700


***** train metrics *****
  epoch                    =        200.0
  total_flos               = 1543153427GF
  train_loss               =       0.3616
  train_runtime            =   6:16:43.38
  train_samples_per_second =         1.77
  train_steps_per_second   =        0.009
{'train_runtime': 22603.3842, 'train_samples_per_second': 1.77, 'train_steps_per_second': 0.009, 'total_flos': 1.656948375552e+18, 'train_loss': 0.36164526638109235, 'epoch': 200.0}


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▂▃▃▄▅▆▆▇████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
train/loss,████▇▇▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,200.0
train/global_step,200
